In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD module and update with correct class names
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Update with your username, password, host, and other credentials
username = "root"
password = "jT0VmTn4mn"
host = "nv-desktop-services.apporto.com"
port = 31432
db = "AAC"
collection = "animals"

# Initialize AnimalShelter with the correct parameters
shelter = AnimalShelter(username, password, host, port, db, collection)

# Retrieve all records from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column to prevent issues with ObjectID in Dash
df.drop(columns=['_id'], inplace=True)

## Debugging: Uncomment these lines if needed
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('CS340_Dashboard')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # Data Table displaying the retrieved MongoDB data
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Setup features for a more user-friendly interface
        page_size=10,  # Display 10 rows per page
        style_table={'overflowX': 'auto'},  # Allow horizontal scrolling
        row_selectable='single',  # Allow single-row selection for map updates
        selected_rows=[0],  # Select first row by default
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }]
    ),
    
    html.Br(),
    html.Hr(),
    
    # Placeholder for the map
    html.Div(id='map-id', className='col s12 m6'),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the geolocation chart based on the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if selected_rows is None:
        row = 0
    else:
        row = selected_rows[0]

    # Convert viewData (a dictionary) to a DataFrame for easy access
    dff = pd.DataFrame.from_dict(viewData)
    
    # If the dataframe is empty, return nothing
    if dff.empty:
        return "No data available."

    # Create a map centered at Austin, TX
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            # Add marker at the animal's location (assuming lat and long are in columns 13 and 14)
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),  # Tooltip shows the breed
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row, 9])  # Popup shows the animal's name
                          ])
                      ])
        ])
    ]

# Run the app inside Jupyter notebook
app.run_server(mode="inline")


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

